In [48]:
#Dataset:
#https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

In [2]:
# import libraries

import boto3, sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [21]:
bucket = 'crime-bucket-2001/Raw/'
data_key = 'Crimes_2001_to_Present.csv'
output_path = 's3://{}/{}/output'.format(bucket, data_key)

In [4]:
!pip install pyspark==3.5.0
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=1620db9d701710b902461f71d8d043a34bd71dae9f57345208ac5b3efc9ce23d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [22]:
!pip install sagemaker_pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 2.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 2.2 MB/s eta 0:00:0000:01
  Created wheel for sagemaker_pyspark: filename=sagemaker_pyspark-1.4.5-py3-none-any.whl size=181610592 sha256=b5bec89bda186807d8e802a0262bdfc89e5421457538d260607068d884124951
  Stored in directory: /root/.cache/pip/wheels/ea/cb/32/140bffbc4ad8465e99b41cd848723a287b07650044483c62fa
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764004 sha256=756b22fae63021ab0566bc35cce8d9fb59534c8f42e798a42dcf1180f29f72e9
  Stored in directory: /root/.cache/pip/wheels/81/9c/6c/d5200fcf351ffa39cbe09911e99703283624cd037df58070d9
Successfully built sagemaker_pyspark pyspark
  Attempting uninstall: py4j
    Found existing install

In [38]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import *
import sagemaker_pyspark
from sagemaker_pyspark import IAMRole

In [55]:
#!/bin/bash

set -eux

sudo yum -y update
sudo yum -y install java-1.8.0-openjdk-devel

SyntaxError: invalid syntax (2672543745.py, line 5)

In [49]:
spark = SparkSession.builder.appName("CrimeInsights").getOrCreate()

JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [40]:
schema = StructType([StructField("ID", IntegerType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", IntegerType(), True),
                            StructField("Community Area", IntegerType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", DoubleType(), True),
                            StructField("Y Coordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [43]:
jars = sagemaker_pyspark.classpath_jars()
print(jars)

['/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-bundle-1.11.901.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-core-1.12.262.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-kms-1.12.262.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-s3-1.12.262.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemaker-1.12.262.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemakerruntime-1.12.262.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sts-1.12.262.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/hadoop-aws-3.3.1.jar', '/opt/conda/lib/python3.10/site-packages/sagemaker_pyspark/jars/sagemaker-spark_2.12-spark_3.3.0-1.4.5.jar']


In [47]:
df = spark.read.csv(output_path)
df

AttributeError: 'Builder' object has no attribute 'read'